In [1]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.5.0


#### _Cell 2_ Για local λύση προβλήματος. Μπορεί να παραβλεφθεί κατά την εκτέλεση του αρχείου, καθώς δεν προσφέρει πειραματικά δεδομένα.

In [2]:
# (θέλουμε τη Java 17 επειδή μόνο οι Java 8 , Java 11 και Java 17 δουλεύουν με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, format_number
from pyspark.sql.types import FloatType
import pandas as pd

Παραδοχή : Χρησιμοποιήσαμε το pandas για να διαβάσουμε τα δεδομένα τα οποία μετατρέψαμε σε CSV και μετά χρησιμοποιήσαμε την spark καθώς δε λειτουργούσε το spark-excel.

In [4]:
# Διαβάζομε τα δεδομένα

tour_occ_ninat = "data/tour_occ_ninat.xlsx"

excel_data = pd.read_excel(tour_occ_ninat, header=8)

In [5]:
# Μετατροπή του xlsx σε csv για να δοθεί στην spark, επειδή δε λειτουργούσε το spark-excel

excel_data.to_csv("data/excel_data.csv", index=False)

In [6]:
# Φτιάχνουμε ένα spark session
spark = SparkSession.builder.appName("EurostatAnalysis").getOrCreate()

In [7]:
# Διαβάζουμε το csv με την spark
spark_df = spark.read.option("header", "true").csv("data/excel_data.csv")

In [8]:
# Ένα print για να δούμε ότι διαβάσαμε σωστά τα δεδομένα
spark_df.show(truncate=False)

+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|GEO/TIME                                        |2006     |2007     |2008     |2009     |2010     |2011     |2012     |2013     |2014     |2015     |
+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Belgium                                         |16039090 |16271311 |16360702 |15451017 |16169676 |16723867 |16432646 |16511721 |17068872 |18852087 |
|Bulgaria                                        |11944694 |12006786 |11791454 |9460922  |10547112 |12426723 |13451440 |14370426 |14077798 |13352281 |
|Czech Republic                                  |20090348 |20610186 |19987022 |17746893 |18365947 |19424839 |21793985 |22144896 |22110112 |23286515 |
|Denmark                                         |9453026  |9327579  |8918197  |8299403  |8981

In [9]:
# Αντικατάσταση του ":" με 0 για να βγουν σωστά οι μέσοι όροι
# και οποιοδήποτε άλλο string γινεται 0
# & μετατροπή των strings σε float για μετέπειτα υπολογισμούς

year_columns = [col for col in spark_df.columns if col.startswith("20")] 
for year_col in year_columns:
    spark_df = spark_df.withColumn(
        year_col,
        when(col(year_col).cast(FloatType()).isNotNull(), col(year_col).cast(FloatType())).otherwise(0.0)
    )

# Ερώτημα 3.1 

In [10]:
# Βρίσκω τον μέσο όρο των στηλών για κάθε χρονο
year_columns = [str(year) for year in range(2006, 2016)]  
average_columns = [col(column) for column in year_columns] 
average_column = sum(average_columns) / len(average_columns)
result = spark_df.select("GEO/TIME", average_column.alias("AverageTourists")) \
    .groupBy("GEO/TIME").avg("AverageTourists")   


In [11]:
result_formatted = result.withColumn("Μέσος Όρος Διανυκτερεύσεων", format_number(col("avg(AverageTourists)"), 2))
result_formatted = result_formatted.drop("avg(AverageTourists)")
# αλλαγή του τίτλου της στήλης GEO/TIME σε Χώρα
result_formatted = result_formatted.withColumnRenamed("GEO/TIME", "Χώρα")

#result_formatted.show(truncate=False)
# ταξινόμηση βάσει ονόματος χώρας
result_formatted = result_formatted.orderBy("GEO/TIME")
result_formatted.toPandas()

,Χώρα,Μέσος Όρος Διανυκτερεύσεων
0,Austria,"74,872,063.20"
1,Belgium,"16,588,099.00"
2,Bulgaria,"12,342,963.60"
3,Croatia,"44,565,893.20"
4,Cyprus,"12,901,906.70"
5,Czech Republic,"20,556,074.40"
6,Denmark,"9,577,326.60"
7,Estonia,"3,398,363.80"
8,Finland,"5,412,151.90"
9,"Former Yugoslav Republic of Macedonia, the","602,361.80"


# Ερώτημα 3.2

In [12]:
def compare_tourist_counts(countries):
    # Αρχικοποιήστε μια λίστα για τα αποτελέσματα
    result = {country: [] for country in countries}

    # Συγκρίνετε τον αριθμό των διανυκτερεύσεων για κάθε επιλεγμένη χώρα έναντι της Ελλάδας
    for country in countries:
        country_df = spark_df.filter(col("GEO/TIME") == country).select(*year_columns).collect()[0]
        greece_df = spark_df.filter(col("GEO/TIME") == "Greece").select(*year_columns).collect()[0]

        for idx, year_col in enumerate(year_columns):
            year_val = int(country_df[idx])
            greece_val = int(greece_df[idx])

            if year_val < greece_val:
                result[country].append(int(year_col))

    return result

In [13]:
# Επιλέχθηκαν οι πρώτες 5 χώρες
selected_countries = ["Belgium", "Bulgaria", "Czech Republic", "Denmark", "Germany (until 1990 former territory of the FRG)"]
result = compare_tourist_counts(selected_countries)

# Εκτύπωση αποτελεσμάτων
for country, years in result.items():
    if len(years) > 0:
        print(f"Για {len(years)} χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα '{country}'. Οι συγκεκριμένες χρονιές: {years}")
    else:
        print(f"Η Ελλάδα δεν είχε ποτέ υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα '{country}'")

Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Belgium'. Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Bulgaria'. Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Czech Republic'. Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Denmark'. Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 2 χρόνια, η Ελλάδα είχε υψηλότερο αριθμό διανυκτερεύσεων από τη χώρα 'Germany (until 1990 former territory of the FRG)'. Οι συγκεκριμένες χρονιές: [2009, 2011]


# Ερώτημα 3.3

In [14]:
def find_countries_with_max_tourists_per_year():
    # Αρχικοποιήστε μια λίστα για τα αποτελέσματα
    result = {}

    # Συγκρίνετε τον αριθμό των διανυκτερεύσεων για κάθε χώρα ανά έτος
    for year_col in year_columns:

        # Βρίσκω την χώρα με τις περισσότερες διανυκτερεύσεις για το συγκεκριμένο έτος
        # και αποθηκεύω το αποτέλεσμα στο dictionary το αποτέλεσμα είναι ένα dictionary με τα έτη ως κλειδιά και τις χώρες με τις περισσότερες διανυκτερεύσεις ως τιμές
        # Π.χ. {'2006': {'GEO/TIME': 'Spain', 'value': 204.0}, '2007': {'GEO/TIME': 'Spain', 'value': 204.0}, ...}
        max_country_row = spark_df.select("GEO/TIME", year_col).orderBy(col(year_col).desc()).limit(1).collect()[0] # Βρίσκω την χώρα με τις περισσότερες διανυκτερεύσεις
        
        max_country = max_country_row["GEO/TIME"] # Παίρνω το όνομα της χώρας
        max_value = max_country_row[year_col] # Παίρνω την τιμή των διανυκτερεύσεων
        # Αποθηκεύω το αποτέλεσμα
        result[year_col] = {"GEO/TIME": max_country, "value": max_value}

    return result

In [15]:
# Καλέστε τη συνάρτηση
result = find_countries_with_max_tourists_per_year()

# Εκτύπωση αποτελεσμάτων
for year_col, data in result.items():
    print(f"Το {year_col} η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η '{data['GEO/TIME']}' με τιμή {data['value']}")


Το 2006 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 224518080.0
Το 2007 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 225450240.0
Το 2008 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 223756224.0
Το 2009 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 200551728.0
Το 2010 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 213349648.0
Το 2011 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 239369168.0
Το 2012 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 243389008.0
Το 2013 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 252447760.0
Το 2014 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 259635792.0
Το 2015 η χώρα με τον μεγαλύτερο αριθμό διανυκτερεύσεων ήταν η 'Spain' με τιμή 269418112.0


# Ερώτημα 3.4

In [16]:
def find_countries_with_min_tourists_per_year():
    # Αρχικοποιήστε μια λίστα για τα αποτελέσματα
    result = {}

    # Συγκρίνετε τον αριθμό των διανυκτερεύσεων για κάθε χώρα ανά έτος
    for year_col in year_columns:
        min_country_row = spark_df.filter(col(year_col) > 0).select("GEO/TIME", year_col).orderBy(col(year_col).asc()).limit(1).collect()[0] # Βρίσκω την χώρα με τις λιγότερες διανυκτερεύσεις
        min_country = min_country_row["GEO/TIME"] # Παίρνω το όνομα της χώρας
        min_value = min_country_row[year_col] # Παίρνω την τιμή των διανυκτερεύσεων
        # Αποθηκεύω το αποτέλεσμα
        result[year_col] = {"GEO/TIME": min_country, "value": min_value}

    return result

In [17]:
# Καλέστε τη συνάρτηση
result = find_countries_with_min_tourists_per_year()

# Εκτύπωση αποτελεσμάτων
# for year_col, data in result.items():
#     print(f"Το {year_col} η χώρα με το μικρότερο αριθμό διανυκτερεύσεων ήταν η '{data['GEO/TIME']}' με τιμή {data['value']}")

# Εναλλακτική εκτύπωση με pandas για καλύτερη παρουσίαση
result_df = pd.DataFrame(result).T
result_df.columns = ["Χώρα", "Διανυκτερεύσεις"]
result_df.index.name = "Έτος"
result_df


,Χώρα,Διανυκτερεύσεις
Έτος,,
2006,Liechtenstein,155171.0
2007,Liechtenstein,167748.0
2008,Liechtenstein,171425.0
2009,Liechtenstein,158414.0
2010,Liechtenstein,150336.0
2011,Liechtenstein,148955.0
2012,Liechtenstein,136276.0
2013,Liechtenstein,132491.0
2014,Liechtenstein,130202.0


In [18]:
# Κλείνουμε την spark session
spark.stop()